In [ ]:
from google.colab import drive
drive.mount("/content/drive/",force_remount=True)

from zipfile import ZipFile
! cp '/content/drive/MyDrive/ML_Capstone/drive-download.zip' '/content'
from zipfile import ZipFile
zip = ZipFile('/content/drive-download.zip')
zip.extractall()

Mounted at /content/drive/


In [ ]:
# importing required packages
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import numpy as np
import cv2
import pickle as pkl
import matplotlib.pyplot as plt
from torchvision import models
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# Set device to cuda if available
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Dataset and Dataloaders

# Lazy Loading

data
-train
  - X
    - 1
      - rgb
        - 0.png
        - 1.png
        - 2.png
      - depth.npy
      - field_id.pkl
    
    - 2

    - ...
  - Y
    - 1.npy
    - 2.npy

-test

In [ ]:
class LazyLoadDataset(Dataset):
  def __init__(self,path,train=True,transform=None):
    self.train = train
    self.transform = transform
    path = path + ("train/" if self.train else "test/")

    self.pathX = path + "X/"
    self.pathY = path + "Y/"

    self.data = os.listdir(self.pathX)

  def __getitem__(self,idx):
    f = self.data[idx]

    # X
    # read rgb images
    img0 = cv2.imread(self.pathX + f + "/rgb/0.png")/255


    if self.transform is not None:
      img0 = self.transform(img0)

    # read field ID
    field_id = pkl.load(open(self.pathX + f + "/field_id.pkl","rb"))

    if self.train==True:
      # Y 
      Y = np.load(self.pathY + f + ".npy")*1000
      return (img0.float().type(torch.float32),field_id),torch.from_numpy(Y).float().type(torch.float32)

    else: # if test, there is no Y
      return img0.float().type(torch.float32),field_id

  def __len__(self):
    return len(self.data)


# Normalization: Compute Mean and Std



In [ ]:
train_dataset_raw = LazyLoadDataset("/content/",train=True,transform=transforms.Compose([
                       transforms.ToTensor(),])) 

In [ ]:
(img_i, id_i),Y_i=train_dataset_raw[0]
img_i.shape

torch.Size([3, 224, 224])

In [ ]:
means=[]
stds=[]
for i in range(3396):
  (img_i, id_i),Y_i=train_dataset_raw[i]
  means.append([torch.mean(img_i[0]).item(),torch.mean(img_i[1]).item(),torch.mean(img_i[2]).item()])
  stds.append([torch.std(img_i[0]).item(),torch.std(img_i[1]).item(),torch.std(img_i[2]).item()])

In [ ]:
img_i

In [ ]:
means

In [ ]:
norm_mean=np.mean(means,axis =0)
norm_mean = list(norm_mean)
norm_mean

[0.3959879955305368, 0.41699021958139393, 0.43522507327886295]

In [ ]:
norm_std=np.mean(stds,axis=0)
norm_std = list(norm_std)
norm_std

[0.21113784175510963, 0.1986706505394656, 0.1991702498400801]

# Train Dataset & Train loader

In [ ]:
#transform=transforms.Compose([transforms.ToTensor()])
train_dataset = LazyLoadDataset("/content/",transform=transforms.Compose([
                       transforms.ToTensor(),
                       #transforms.RandomRotation(degrees=(0, 30)),
                       #transforms.ColorJitter(brightness=0.5, contrast=1, saturation=0.1, hue=0.5),
                       #transforms.RandomPerspective(distortion_scale=0.5, p=0.4),
                       transforms.Normalize(norm_mean, norm_std)])) 

In [ ]:
(img_0, field_id), Y = train_dataset[0]

In [ ]:
img_0.shape

torch.Size([3, 224, 224])

In [ ]:
batch_size_train = 2
train_loader=DataLoader(train_dataset,batch_size=batch_size_train,shuffle=True)

In [ ]:
Y/1000

tensor([ 0.0522,  0.0528,  0.1172,  0.0645, -0.0028,  0.0988,  0.0699, -0.0515,
         0.0879,  0.0538,  0.0143, -0.0535])

# Test Dataset & Test loader

In [ ]:
test_dataset = LazyLoadDataset("/content/",train=False,transform=transforms.Compose([
                       transforms.ToTensor(),
                       #transforms.RandomRotation(degrees=(0, 30)),
                       #transforms.ColorJitter(brightness=0.5, contrast=1, saturation=0.1, hue=0.5),
                       #transforms.RandomPerspective(distortion_scale=0.5, p=0.4),
                       transforms.Normalize(norm_mean, norm_std),
                   ])) 

batch_size_test = 1
test_loader=DataLoader(test_dataset,batch_size=batch_size_test,shuffle=True)

# Model

In [ ]:
class Res(nn.Module):
    def __init__(self, input_channels, output_size):
        super(Res, self).__init__()
        
        # We use Sequential for simplicity
        self.stack = nn.Sequential(
            models.resnet50(pretrained=True),
            nn.Linear(1000, output_size)
        )                               
    def forward(self, x):
        x = self.stack(x)
        return x

In [ ]:
# Model input/output settings
input_channels = 3 # number of input channels
output_size=12

In [ ]:
def train(epoch, model, optimizer,loader=train_loader):
    """
    Train the model for one epoch

    Args:
        epoch (int): current epoch
        model (nn.Module): model to train
        optimizer (torch.optim): optimizer to use
    """
    model.train()

    for batch_idx,((data, id),target) in enumerate(loader):
        # send to device
        data, target = data.to(device), target.to(device)

        # consider passing different data augmentation in training
        # data = F.invert()
        # data = F.adjust_sharpness(data,sharpness_factor = 0.5)
        # data = F.adjust_hue(data,hue_factor = 0.9)
        # data = F.adjust_saturation(data,saturation_factor = 0.5)
        
        data = data.view(len(id),3,224, 224)

        # make sure we erase all the gradients before computing new ones
        optimizer.zero_grad() 
        
        # forward propagation
        output = model(data)

        #RMSE loss
        loss = torch.sqrt(nn.functional.mse_loss(output,target))
        
        # backward propagation
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss))
    return loss

In [ ]:
def test(model):
    """
    Test the model

    Args:
        model (nn.Module): model to test
      
    """
    model.eval()
    ids = []
    preds = []
    with torch.no_grad():
      for batch_idx,(data, id) in enumerate(test_loader):
        # send to device
        data = data.to(device)          
        pred = model(data)
        for i in range(len(id)):
          ids.append(id[i])
          preds.append(np.array(pred[i].cpu()/1000,dtype="float64"))

    return ids, preds  

# Cross-Validation & Grid-Search for determining hyperparameters

What should hyperparameters in the models be? We could perform grid-search on the whole train dataset and pick the best-performing hyerparameters. But to avoid over-fitting on the train dataset, we split the whole train dataset into 9:1.
*   cross_val_train_set (90% of train dataset)
*   cross_val_test_set (10% of train dataset)

We train our model on the cross_val_train_set and test our model on cross_val_test_set. We pick the best-performing hyperparameters on the cross_val_test_set.

After all these steps, we train our model using the picked hyperparameters on the whole train dataset. 

In [ ]:
cross_val_train_set, cross_val_test_set = train_test_split(train_dataset, test_size=0.1)

In [ ]:
# Sanity Check
len(cross_val_train_set) == int(len(train_dataset)*0.9)

True

In [ ]:
batch_size_cross_val_train = 4
batch_size_cross_val_test = len(cross_val_test_set) 

In [ ]:
cross_val_train_loader=DataLoader(cross_val_train_set,batch_size=batch_size_cross_val_train,shuffle=True)
cross_val_test_loader=DataLoader(cross_val_test_set,batch_size=batch_size_cross_val_test,shuffle=True)

In [ ]:
len(cross_val_test_loader)

1

In [ ]:
def cross_val(model):
    """
    Test the model

    Args:
        model (nn.Module): model to test
      
    """
    model.eval()
    # We can just edit on the code for train(). We should delete the bakward propagation part and optimizer part.

    with torch.no_grad():
      for batch_idx,((data, id),target) in enumerate(cross_val_test_loader):
          # send to device
          if batch_idx==0:
            data, target = data.to(device), target.to(device)

            # forward propagation
            output = model(data)

            #RMSE loss
            loss = torch.sqrt(nn.functional.mse_loss(output,target))
            # Since batch_size_cross_val_test = len(cross_val_test_set), we have only one batch. So we can just return the loss on this batch.
            return loss.item()

In [ ]:
def SGD_hyper_tuning(lr_list,momentum_list):  
  for j in momentum_list:
    for i in lr_list:
      model_res = Res(input_channels, output_size) # create Res model
      model_res.to(device)
      print("lr=",i,"momentum=",j)
      train(epoch=0, model=model_res, optimizer=torch.optim.SGD(model_res.parameters(), lr=i,momentum=j),loader=cross_val_train_loader)
      print("Validation Performance:",cross_val(model_res))

In [ ]:
lr_list = [1e-5,1e-4,1e-3,1e-2,1e-1]
momentum_list = [0.99,0.97,0.95,0.93,0.91]
SGD_hyper_tuning(lr_list,momentum_list)

lr= 1e-05 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 69.606239
Train Epoch: 0 [800/3056 (26%)]	Loss: 60.257511
Train Epoch: 0 [1600/3056 (52%)]	Loss: 23.980026
Train Epoch: 0 [2400/3056 (79%)]	Loss: 14.367318
Validation Performance: 13.263710975646973
lr= 0.0001 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 64.843323
Train Epoch: 0 [800/3056 (26%)]	Loss: 20.624750
Train Epoch: 0 [1600/3056 (52%)]	Loss: 10.833291
Train Epoch: 0 [2400/3056 (79%)]	Loss: 9.235901
Validation Performance: 7.795881271362305
lr= 0.001 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 66.968620
Train Epoch: 0 [800/3056 (26%)]	Loss: 16.725407
Train Epoch: 0 [1600/3056 (52%)]	Loss: 10.579671
Train Epoch: 0 [2400/3056 (79%)]	Loss: 10.512009
Validation Performance: 10.428595542907715
lr= 0.01 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 69.055740
Train Epoch: 0 [800/3056 (26%)]	Loss: 23.698336
Train Epoch: 0 [1600/3056 (52%)]	Loss: 16.356617
Train Epoch: 0 [2400/3056 (79%)]	Loss: 18.632277
Validatio



Top 3 Performance
1.  lr = 1e-3, momentum = 0.91, loss on validation set = 5.897
2.  lr = 1e-3, momentum = 0.93, loss on validation set = 6.93
3.  lr = 1e-3, momentum = 0.95, loss on validation set = 6.94





In [ ]:
lr_list = [8e-4,9e-4,1e-3,2e-3,3e-3]
momentum_list = [0.99,0.97,0.95,0.93,0.91]
SGD_hyper_tuning(lr_list,momentum_list)

lr= 0.0008 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 68.818230
Train Epoch: 0 [800/3056 (26%)]	Loss: 18.830072
Train Epoch: 0 [1600/3056 (52%)]	Loss: 17.464287
Train Epoch: 0 [2400/3056 (79%)]	Loss: 11.113073
Validation Performance: 13.585453987121582
lr= 0.0009 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 67.563614
Train Epoch: 0 [800/3056 (26%)]	Loss: 28.579670
Train Epoch: 0 [1600/3056 (52%)]	Loss: 16.316357
Train Epoch: 0 [2400/3056 (79%)]	Loss: 11.917153
Validation Performance: 13.811532020568848
lr= 0.001 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 67.213844
Train Epoch: 0 [800/3056 (26%)]	Loss: 15.255556
Train Epoch: 0 [1600/3056 (52%)]	Loss: 11.603078
Train Epoch: 0 [2400/3056 (79%)]	Loss: 9.428789
Validation Performance: 10.829007148742676
lr= 0.002 momentum= 0.99
Train Epoch: 0 [0/3056 (0%)]	Loss: 69.599236
Train Epoch: 0 [800/3056 (26%)]	Loss: 15.523223
Train Epoch: 0 [1600/3056 (52%)]	Loss: 14.475142
Train Epoch: 0 [2400/3056 (79%)]	Loss: 14.936378
Valida

Top 5 Performance

1.   lr = 3e-3, momentum = 0.91, loss on validation set = 5.72
1.   lr = 8e-4, momentum = 0.95, loss on validation set = 5.75
2.   lr = 9e-4, momentum = 0.91, loss on validation set = 5.82

3.   lr = 9e-4, momentum = 0.95, loss on validation set = 6.25

4.   lr = 1e-3, momentum = 0.95, loss on validation set = 6.56

#lr=3e-3, momentum=0.91, 10 epoch, train_batch_size = 2, with lr decay

In [ ]:
model_res = Res(input_channels, output_size) # create Res model
model_res.to(device)
optimizer = torch.optim.SGD(model_res.parameters(), lr=3e-3,momentum=0.91) 
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

print("lr=",3e-3,"momentum=",0.91)
for epoch in range(10):
  train(epoch, model=model_res, optimizer=torch.optim.SGD(model_res.parameters(), lr=3e-3,momentum=0.91),loader=train_loader)
  scheduler.step()

lr= 0.003 momentum= 0.91
Train Epoch: 0 [0/3396 (0%)]	Loss: 67.939072
Train Epoch: 0 [400/3396 (12%)]	Loss: 13.923128
Train Epoch: 0 [800/3396 (24%)]	Loss: 13.405359
Train Epoch: 0 [1200/3396 (35%)]	Loss: 9.460212
Train Epoch: 0 [1600/3396 (47%)]	Loss: 10.889026
Train Epoch: 0 [2000/3396 (59%)]	Loss: 8.984818
Train Epoch: 0 [2400/3396 (71%)]	Loss: 10.970617
Train Epoch: 0 [2800/3396 (82%)]	Loss: 8.084895
Train Epoch: 0 [3200/3396 (94%)]	Loss: 6.002130


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Train Epoch: 1 [0/3396 (0%)]	Loss: 8.923368
Train Epoch: 1 [400/3396 (12%)]	Loss: 6.965173
Train Epoch: 1 [800/3396 (24%)]	Loss: 5.476079
Train Epoch: 1 [1200/3396 (35%)]	Loss: 7.721508
Train Epoch: 1 [1600/3396 (47%)]	Loss: 4.278842
Train Epoch: 1 [2000/3396 (59%)]	Loss: 6.796082
Train Epoch: 1 [2400/3396 (71%)]	Loss: 7.104254
Train Epoch: 1 [2800/3396 (82%)]	Loss: 5.075162
Train Epoch: 1 [3200/3396 (94%)]	Loss: 9.091908
Train Epoch: 2 [0/3396 (0%)]	Loss: 4.974702
Train Epoch: 2 [400/3396 (12%)]	Loss: 3.828071
Train Epoch: 2 [800/3396 (24%)]	Loss: 4.713331
Train Epoch: 2 [1200/3396 (35%)]	Loss: 5.700161
Train Epoch: 2 [1600/3396 (47%)]	Loss: 6.056094
Train Epoch: 2 [2000/3396 (59%)]	Loss: 4.684329
Train Epoch: 2 [2400/3396 (71%)]	Loss: 4.639878
Train Epoch: 2 [2800/3396 (82%)]	Loss: 3.255405
Train Epoch: 2 [3200/3396 (94%)]	Loss: 3.079749
Train Epoch: 3 [0/3396 (0%)]	Loss: 4.688659
Train Epoch: 3 [400/3396 (12%)]	Loss: 2.683493
Train Epoch: 3 [800/3396 (24%)]	Loss: 4.055069
Train Epoc

In [ ]:
torch.save(model_res,'/content/drive/MyDrive/ML_Capstone/v4_res_3img0_m2.pt')
model_res = torch.load('/content/drive/MyDrive/ML_Capstone/v4_res_3img0_m2.pt')
model_res.eval()
ids,preds = test(model_res)



outfile = '/content/drive/MyDrive/ML_Capstone/submission_v4_res_3img0_m2.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']

df = pd.concat([pd.DataFrame(ids), pd.DataFrame.from_records(preds)], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

# lr=3e-3, momentum=0.91, 20 epoch, train_batch_size = 2, no lr decay

In [ ]:
model_res = Res(input_channels, output_size) # create Res model
model_res.to(device)
optimizer = torch.optim.SGD(model_res.parameters(), lr=3e-3,momentum=0.91) 

print("lr=",3e-3,"momentum=",0.91)
for epoch in range(20):
  train(epoch, model=model_res, optimizer=torch.optim.SGD(model_res.parameters(), lr=3e-3,momentum=0.91),loader=train_loader)

lr= 0.003 momentum= 0.91
Train Epoch: 0 [0/3396 (0%)]	Loss: 65.925781
Train Epoch: 0 [400/3396 (12%)]	Loss: 24.333570
Train Epoch: 0 [800/3396 (24%)]	Loss: 9.356822
Train Epoch: 0 [1200/3396 (35%)]	Loss: 10.962144
Train Epoch: 0 [1600/3396 (47%)]	Loss: 11.152804
Train Epoch: 0 [2000/3396 (59%)]	Loss: 12.381385
Train Epoch: 0 [2400/3396 (71%)]	Loss: 10.082131
Train Epoch: 0 [2800/3396 (82%)]	Loss: 9.438674
Train Epoch: 0 [3200/3396 (94%)]	Loss: 6.253980
Train Epoch: 1 [0/3396 (0%)]	Loss: 8.764084
Train Epoch: 1 [400/3396 (12%)]	Loss: 6.879309
Train Epoch: 1 [800/3396 (24%)]	Loss: 4.917552
Train Epoch: 1 [1200/3396 (35%)]	Loss: 6.809510
Train Epoch: 1 [1600/3396 (47%)]	Loss: 8.005464
Train Epoch: 1 [2000/3396 (59%)]	Loss: 7.180151
Train Epoch: 1 [2400/3396 (71%)]	Loss: 5.265434
Train Epoch: 1 [2800/3396 (82%)]	Loss: 7.302955
Train Epoch: 1 [3200/3396 (94%)]	Loss: 4.902833
Train Epoch: 2 [0/3396 (0%)]	Loss: 7.030436
Train Epoch: 2 [400/3396 (12%)]	Loss: 6.550316
Train Epoch: 2 [800/3396 (

In [ ]:
torch.save(model_res,'/content/drive/MyDrive/ML_Capstone/v4_res_3img0_m3.pt')
model_res = torch.load('/content/drive/MyDrive/ML_Capstone/v4_res_3img0_m3.pt')
model_res.eval()
ids,preds = test(model_res)

import pickle
import pandas as pd

outfile = '/content/drive/MyDrive/ML_Capstone/submission_v4_res_3img0_m3.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']

df = pd.concat([pd.DataFrame(ids), pd.DataFrame.from_records(preds)], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

Conclusion for SGD_hyperparameter choosing: 
When lr=3e-3, momentum=0.91, 20 epoch, train_batch_size = 2, no lr decay, we get best score on Kaggle.

In [ ]:
def Adam_hyper_tuning(lr_list):  
  for i in lr_list:
    model_res = Res(input_channels, output_size) # create Res model
    model_res.to(device)
    print("lr=",i)
    train(epoch=0, model=model_res, optimizer=torch.optim.Adam(model_res.parameters(), lr=i),loader=cross_val_train_loader)
    print("Validation Performance:",cross_val(model_res))

In [ ]:
lr_list = [1e-5,1e-4,1e-3,1e-2,1e-1]
Adam_hyper_tuning(lr_list)

lr= 1e-05
Train Epoch: 0 [0/3056 (0%)]	Loss: 65.144081
Train Epoch: 0 [800/3056 (26%)]	Loss: 49.260761
Train Epoch: 0 [1600/3056 (52%)]	Loss: 21.110296
Train Epoch: 0 [2400/3056 (79%)]	Loss: 18.653252
Validation Performance: 13.924497604370117
lr= 0.0001
Train Epoch: 0 [0/3056 (0%)]	Loss: 64.287918
Train Epoch: 0 [800/3056 (26%)]	Loss: 14.503418
Train Epoch: 0 [1600/3056 (52%)]	Loss: 9.274224
Train Epoch: 0 [2400/3056 (79%)]	Loss: 12.771055
Validation Performance: 8.159941673278809
lr= 0.001
Train Epoch: 0 [0/3056 (0%)]	Loss: 66.451294
Train Epoch: 0 [800/3056 (26%)]	Loss: 14.505063
Train Epoch: 0 [1600/3056 (52%)]	Loss: 11.473140
Train Epoch: 0 [2400/3056 (79%)]	Loss: 6.911225
Validation Performance: 8.506036758422852
lr= 0.01
Train Epoch: 0 [0/3056 (0%)]	Loss: 66.403252
Train Epoch: 0 [800/3056 (26%)]	Loss: 20.115875
Train Epoch: 0 [1600/3056 (52%)]	Loss: 29.106987
Train Epoch: 0 [2400/3056 (79%)]	Loss: 22.614956
Validation Performance: 26.6864013671875
lr= 0.1
Train Epoch: 0 [0/3056

Top 2 Performance
1.  lr = 1e-4,  loss on validation set = 8.15
2.  lr = 1e-3,  loss on validation set = 8.5

In [ ]:
lr_list = [1e-4,2e-4,3e-4,4e-4,5e-4]
Adam_hyper_tuning(lr_list)

lr= 0.0001
Train Epoch: 0 [0/3056 (0%)]	Loss: 66.667213
Train Epoch: 0 [800/3056 (26%)]	Loss: 13.924146
Train Epoch: 0 [1600/3056 (52%)]	Loss: 8.433039
Train Epoch: 0 [2400/3056 (79%)]	Loss: 8.123702
Validation Performance: 6.761453628540039
lr= 0.0002
Train Epoch: 0 [0/3056 (0%)]	Loss: 63.493702
Train Epoch: 0 [800/3056 (26%)]	Loss: 12.982480
Train Epoch: 0 [1600/3056 (52%)]	Loss: 7.902048
Train Epoch: 0 [2400/3056 (79%)]	Loss: 9.107423
Validation Performance: 7.139166355133057
lr= 0.0003
Train Epoch: 0 [0/3056 (0%)]	Loss: 68.070793
Train Epoch: 0 [800/3056 (26%)]	Loss: 10.867259
Train Epoch: 0 [1600/3056 (52%)]	Loss: 11.598380
Train Epoch: 0 [2400/3056 (79%)]	Loss: 8.672360
Validation Performance: 6.6652913093566895
lr= 0.0004
Train Epoch: 0 [0/3056 (0%)]	Loss: 66.768654
Train Epoch: 0 [800/3056 (26%)]	Loss: 11.016487
Train Epoch: 0 [1600/3056 (52%)]	Loss: 9.423256
Train Epoch: 0 [2400/3056 (79%)]	Loss: 6.551960
Validation Performance: 7.405411243438721
lr= 0.0005
Train Epoch: 0 [0/3

Top 1 Performance
1.  lr = 3e-4,  loss on validation set = 6.66

In [ ]:
model_res = Res(input_channels, output_size) # create Res model
model_res.to(device)
optimizer = torch.optim.Adam(model_res.parameters(), lr=3e-4) 

print("lr=",3e-4)
for epoch in range(20):
  train(epoch, model=model_res, optimizer=torch.optim.Adam(model_res.parameters(), lr=3e-4),loader=train_loader)

lr= 0.0003
Train Epoch: 0 [0/3396 (0%)]	Loss: 68.487396
Train Epoch: 0 [400/3396 (12%)]	Loss: 18.962946
Train Epoch: 0 [800/3396 (24%)]	Loss: 16.527212
Train Epoch: 0 [1200/3396 (35%)]	Loss: 13.674988
Train Epoch: 0 [1600/3396 (47%)]	Loss: 10.032792
Train Epoch: 0 [2000/3396 (59%)]	Loss: 9.797058
Train Epoch: 0 [2400/3396 (71%)]	Loss: 5.768251
Train Epoch: 0 [2800/3396 (82%)]	Loss: 6.646792
Train Epoch: 0 [3200/3396 (94%)]	Loss: 4.450601
Train Epoch: 1 [0/3396 (0%)]	Loss: 7.412120
Train Epoch: 1 [400/3396 (12%)]	Loss: 6.709545
Train Epoch: 1 [800/3396 (24%)]	Loss: 6.022833
Train Epoch: 1 [1200/3396 (35%)]	Loss: 7.240652
Train Epoch: 1 [1600/3396 (47%)]	Loss: 4.797465
Train Epoch: 1 [2000/3396 (59%)]	Loss: 5.765735
Train Epoch: 1 [2400/3396 (71%)]	Loss: 5.188417
Train Epoch: 1 [2800/3396 (82%)]	Loss: 5.439257
Train Epoch: 1 [3200/3396 (94%)]	Loss: 5.257957
Train Epoch: 2 [0/3396 (0%)]	Loss: 7.138648
Train Epoch: 2 [400/3396 (12%)]	Loss: 4.166456
Train Epoch: 2 [800/3396 (24%)]	Loss: 4.7

In [ ]:
torch.save(model_res,'/content/drive/MyDrive/ML_Capstone/v4_res_3img0_m4.pt')
model_res = torch.load('/content/drive/MyDrive/ML_Capstone/v4_res_3img0_m4.pt')
model_res.eval()
ids,preds = test(model_res)

import pickle
import pandas as pd

outfile = '/content/drive/MyDrive/ML_Capstone/submission_v4_res_3img0_m4.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']

df = pd.concat([pd.DataFrame(ids), pd.DataFrame.from_records(preds)], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

Conclusion for Adam_hyperparameter choosing: Although from results here, ADAM with best parameters perform better than SGD, on Kaggle, SGD still wins over ADAM. Maybe it's because Kaggle only uses 60% of testing data or overfitting problem.